# Data processing

In [1]:
import boto3
import os
import re
import pickle
import json
import pgeocode

import pandas as pd
import numpy as np


from geopy.geocoders import Nominatim

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn import metrics

from src.utils.general import get_s3_credentials, read_pkl_from_s3
from src.pipeline.preprocessing import df_to_lower_case, change_misspelled_chicago_city_names, convert_nan, transform_label
from src.pipeline.feature_engineering import transformate_and_aggregate_dates, remove_non_useful_columns, aggregate_num_violations
from src.utils.constants import L

In [2]:
s3_creds = get_s3_credentials("../../conf/local/credentials.yaml")

INFO:root:Leyendo las credenciales de ../../conf/local/credentials.yaml


In [3]:
session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

In [4]:
bucket='data-product-architecture-equipo-3'

In [5]:
objects = s3.list_objects_v2(Bucket=bucket)['Contents']

In [6]:
for file in objects:
    print(file['Key'])

ingestion/consecutive/YEAR=2020/MONTH=11/consecutive-inspections-2020-11-03.pkl
ingestion/consecutive/YEAR=2021/MONTH=3/consecutive-inspections-2021-03-01.pkl
ingestion/consecutive/YEAR=2021/MONTH=3/consecutive-inspections-2021-03-08.pkl
ingestion/consecutive/YEAR=2021/MONTH=3/consecutive-inspections-2021-03-15.pkl
ingestion/initial/YEAR=2021/MONTH=2/historic-inspections-2021-02-23.pkl


In [7]:
# Para obtener el path
os.path.dirname('ingestion/initial/YEAR=2021/MONTH=2/historic-inspections-2021-02-23.pkl')

'ingestion/initial/YEAR=2021/MONTH=2'

In [8]:
# Para obtener el nombre del archivo
filename = os.path.basename('ingestion/initial/YEAR=2021/MONTH=2/historic-inspections-2021-02-23.pkl')
filename

'historic-inspections-2021-02-23.pkl'

In [9]:
# Para extraer la fecha del filename
re.findall(r'[ :]*\d+[ -/]\d+[ -/]\d+', filename, re.IGNORECASE)

['2021-02-23']

Por ahora sólo leeremos el inicial

In [10]:
filename="ingestion/initial/YEAR=2021/MONTH=2/historic-inspections-2021-02-23.pkl"
print("Leyendo {}...".format(filename))
json_file = read_pkl_from_s3(s3, bucket, filename)

Leyendo ingestion/initial/YEAR=2021/MONTH=2/historic-inspections-2021-02-23.pkl...


In [11]:
# Convertimos a dataframe
food_inspection_df = pd.DataFrame(json_file)
food_inspection_df.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2492939,JACK AND GINGERS,JACK AND GINGERS,1767508,Restaurant,Risk 1 (High),2048 W ARMITAGE AVE,CHICAGO,IL,60647,2021-03-16T00:00:00.000,Non-Inspection,No Entry,41.917829917196265,-87.67953075090323,"{'latitude': '-87.67953075090323', 'longitude'...",NaN
1,2492937,Hanson Park Branch,Hanson Park Branch,24462,School,Risk 1 (High),2318 N Lorel ST,CHICAGO,IL,60639,2021-03-16T00:00:00.000,Canvass,Pass,41.922831144138776,-87.75994374773492,"{'latitude': '-87.75994374773492', 'longitude'...","55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ..."
2,2492907,WHIPPED & BEATEN CULINARY WORKS,WHIPPED & BEATEN CULINARY WORKS,2776158,COOKING SCHOOL,Risk 3 (Low),3611-3617 N BROADWAY,CHICAGO,IL,60613,2021-03-16T00:00:00.000,License,Pass,41.94773174730859,-87.64734303203468,"{'latitude': '-87.64734303203468', 'longitude'...",NaN
3,2492904,WINTHROP CHILDREN,WNTHROP CHILDREN,2215541,Daycare (2 - 6 Years),Risk 1 (High),4848 N WINTHROP AVE,CHICAGO,IL,60640,2021-03-16T00:00:00.000,Canvass,Out of Business,41.97048400251602,-87.65780618896973,"{'latitude': '-87.65780618896973', 'longitude'...",NaN
4,2492841,BAKER MILLER,BAKER MILLER,2616224,Restaurant,Risk 1 (High),4655 N LINCOLN AVE,CHICAGO,IL,60625,2021-03-15T00:00:00.000,Canvass Re-Inspection,Pass,41.96641308847062,-87.68693718914149,"{'latitude': '-87.68693718914149', 'longitude'...",40. PERSONAL CLEANLINESS - Comments: 2-402.11 ...


In [15]:
food_inspection_df[food_inspection_df['dba_name']=='WHIPPED & BEATEN CULINARY WORKS']

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
2,2492907,WHIPPED & BEATEN CULINARY WORKS,WHIPPED & BEATEN CULINARY WORKS,2776158,COOKING SCHOOL,Risk 3 (Low),3611-3617 N BROADWAY,CHICAGO,IL,60613,2021-03-16T00:00:00.000,License,Pass,41.94773174730859,-87.64734303203468,"{'latitude': '-87.64734303203468', 'longitude'...",NaN
426,2492292,WHIPPED & BEATEN CULINARY WORKS,WHIPPED & BEATEN CULINARY WORKS,2590079,COOKING SCHOOL,Risk 1 (High),3611-3617 N BROADWAY,CHICAGO,IL,60613,2021-03-02T00:00:00.000,Canvass,Pass,41.94773174730859,-87.64734303203468,"{'latitude': '-87.64734303203468', 'longitude'...",NaN
8548,2385216,WHIPPED & BEATEN CULINARY WORKS,WHIPPED & BEATEN CULINARY WORKS,2590079,COOKING SCHOOL,Risk 1 (High),3611-3617 N BROADWAY,CHICAGO,IL,60613,2020-07-22T00:00:00.000,Canvass,Pass w/ Conditions,41.94773174730859,-87.64734303203468,"{'latitude': '-87.64734303203468', 'longitude'...",16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED...
29180,2290513,WHIPPED & BEATEN CULINARY WORKS,WHIPPED & BEATEN CULINARY WORKS,2590079,COOKING SCHOOL,Risk 1 (High),3611-3617 N BROADWAY,CHICAGO,IL,60613,2019-05-28T00:00:00.000,Canvass,Pass w/ Conditions,41.94773174730859,-87.64734303203468,"{'latitude': '-87.64734303203468', 'longitude'...","3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."
44529,2182165,WHIPPED & BEATEN CULINARY WORKS,WHIPPED & BEATEN CULINARY WORKS,2590079,COOKING SCHOOL,Risk 1 (High),3611-3617 N BROADWAY,CHICAGO,IL,60613,2018-06-29T00:00:00.000,License Re-Inspection,Pass,41.94773174730859,-87.64734303203468,"{'latitude': '-87.64734303203468', 'longitude'...",32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...
44809,2181902,WHIPPED & BEATEN CULINARY WORKS,WHIPPED & BEATEN CULINARY WORKS,2590079,COOKING SCHOOL,Risk 1 (High),3611-3617 N BROADWAY,CHICAGO,IL,60613,2018-06-22T00:00:00.000,License,Fail,41.94773174730859,-87.64734303203468,"{'latitude': '-87.64734303203468', 'longitude'...","11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D..."


In [17]:
food_inspection_df[food_inspection_df['license_']=='2776158']

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
2,2492907,WHIPPED & BEATEN CULINARY WORKS,WHIPPED & BEATEN CULINARY WORKS,2776158,COOKING SCHOOL,Risk 3 (Low),3611-3617 N BROADWAY,CHICAGO,IL,60613,2021-03-16T00:00:00.000,License,Pass,41.94773174730859,-87.64734303203468,"{'latitude': '-87.64734303203468', 'longitude'...",NaN


In [21]:
len(food_inspection_df['zip'].unique())

113

In [20]:
food_inspection_df['results'].unique()

array(['No Entry', 'Pass', 'Out of Business', 'Pass w/ Conditions',
       'Fail', 'Not Ready', 'Business Not Located'], dtype=object)

# Preprocesamiento

In [181]:
food_df = df_to_lower_case(food_inspection_df)

In [182]:
# Creo que esta parte se puede hacer mejor en sql
food_df = change_misspelled_chicago_city_names(food_df)

In [183]:
# Creo que esta parte se puede hacer mejor en sql
food_df = convert_nan(food_df)

In [ ]:
food_df = transform_label(food_df)

# Feature Engineering

In [186]:
food_df = transformate_and_aggregate_dates(L, food_df)

/home/user/Documents/Caro_MCD/DPA/food_inspection_DPA/src/pipeline/feature_engineering.py:26: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  date_gen = (getattr(df['inspection_date'].dt, i).rename(i) for i in lista)


In [187]:
food_df = aggregate_num_violations(food_df)

In [188]:
food_df = remove_non_useful_columns(food_df)

In [189]:
food_df

,license_,facility_type,risk,city,state,zip,inspection_type,results,latitude,longitude,year,month,day,dayofweek,dayofyear,week,quarter,num_violations
0,1767508,restaurant,high,chicago,il,60647,non-inspection,no entry,41.917829917196265,-87.67953075090323,2021,3,16,1,75,11,1,0
1,24462,school,high,chicago,il,60639,canvass,pass,41.922831144138776,-87.75994374773492,2021,3,16,1,75,11,1,1
2,2776158,cooking school,low,chicago,il,60613,license,pass,41.94773174730859,-87.64734303203468,2021,3,16,1,75,11,1,0
3,2215541,daycare (2 - 6 years),high,chicago,il,60640,canvass,out of business,41.97048400251602,-87.65780618896973,2021,3,16,1,75,11,1,0
4,2616224,restaurant,high,chicago,il,60625,canvass re-inspection,pass,41.96641308847062,-87.68693718914149,2021,3,15,0,74,11,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217404,1992040,restaurant,high,chicago,il,60613,license re-inspection,pass,41.961605669949854,-87.67596676683779,2010,1,4,0,4,1,1,0
217405,2013328,restaurant,high,chicago,il,60657,license re-inspection,pass,41.938006880423615,-87.6447545707008,2010,1,4,0,4,1,1,0
217406,1899292,restaurant,high,chicago,il,60634,license re-inspection,pass,41.93844282365204,-87.76831838068422,2010,1,4,0,4,1,1,0
217407,1992039,restaurant,high,chicago,il,60613,license re-inspection,pass,41.961605669949854,-87.67596676683779,2010,1,4,0,4,1,1,0


In [190]:
food_df = remove_nan_rows(food_df)

In [197]:
import logging
def feature_generation(df):
    #df = transformate_and_aggregate_dates(df)
    #df = aggregate_num_violations(df)

    # Aplicamos OneHot Encoder para las categóricas
    transformers = [('one_hot', OneHotEncoder(), ['facility_type', 'risk',
                                                  'city', 'state', 'inspection_type'])]

    col_trans = ColumnTransformer(transformers, remainder="passthrough", n_jobs=-1)

    # Ordenaremos el dataframe temporalmente
    df = df.sort_values(by=["year", "month", "day"])

    X = col_trans.fit_transform(df.drop(columns="label"))
    y = df['label'].values.reshape(X.shape[0], )
    logging.info("Successfully transformation of the discrete variables.'")

    logging.info("Converting to dataframe...")
    X = X.todense()
    df = pd.DataFrame(X, columns=col_trans.get_feature_names())
    df['label'] = y

    return df

In [222]:
def feature_selection(df):

    # Separación en train y test manualmente para no hacer data leaking
    lim = round(df.shape[0] * .70)  # 70% de train
    X_train, X_test = df[:lim].drop('label', axis=1), df[lim:].drop('label', axis=1)
    y_train, y_test = df[['label']][:lim], df[['label']][lim:]

    return  X_train, X_test, y_train, y_test

In [199]:
df, X, y = feature_generation(food_df)

INFO:root:Successfully transformation of the discrete variables.'
INFO:root:Converting to dataframe...


In [223]:
X_train, X_test, y_train, y_test = feature_selection(df)

In [227]:
X_train

,one_hot__x0_(convenience store),one_hot__x0_(gas station),one_hot__x0_1005 nursing home,one_hot__x0_1023,one_hot__x0_1023 childern's service facility,one_hot__x0_1023 childern's service s facility,one_hot__x0_1023 childern's services facility,one_hot__x0_1023 children's services facility,one_hot__x0_1023-children's services facility,one_hot__x0_1475 liquor,...,latitude,longitude,year,month,day,dayofweek,dayofyear,week,quarter,num_violations
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.884586,-87.631010,2010.0,1.0,4.0,0.0,4.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.961606,-87.675967,2010.0,1.0,4.0,0.0,4.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.938007,-87.644755,2010.0,1.0,4.0,0.0,4.0,1.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.938443,-87.768318,2010.0,1.0,4.0,0.0,4.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.961606,-87.675967,2010.0,1.0,4.0,0.0,4.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.951113,-87.663982,2017.0,6.0,16.0,4.0,167.0,24.0,2.0,2.0
151670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.785733,-87.738438,2017.0,6.0,16.0,4.0,167.0,24.0,2.0,2.0
151671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.921981,-87.644132,2017.0,6.0,16.0,4.0,167.0,24.0,2.0,11.0
151672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.892249,-87.609518,2017.0,6.0,16.0,4.0,167.0,24.0,2.0,4.0


# Exploración

# Buscamos dónde hay nulos y ver cómo los imputamos 

In [29]:
columna = 'license_'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

17

# Facility type

In [31]:
columna = 'facility_type'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

4897

In [197]:
# imputamos facility type con Unkown
food_inspection_df.loc[bool_series, 'facility_type'] = "unkown"


# Risk

In [202]:
columna = 'risk'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

69

In [209]:
# Cambiamos nombres
food_inspection_df.loc[food_inspection_df['risk'] == 'risk 1 (high)', 'risk'] = "high"
food_inspection_df.loc[food_inspection_df['risk'] == 'risk 2 (medium)', 'risk'] = "medium"
food_inspection_df.loc[food_inspection_df['risk'] == 'risk 3 (low)', 'risk'] = "low"
food_inspection_df.loc[bool_series, 'risk'] = 'unkown_risk'

# Address

In [24]:
columna = 'address'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

0

In [228]:
food_inspection_df['address'][0].strip().split(' ')[-1]

'ave'

In [242]:
food_inspection_df['street_type'] = 'unknown street type'

In [234]:
food_inspection_df.street_type = food_inspection_df.applymap(lambda s:s.strip().split(' ')[-1] if type(s) == str else s)

In [250]:
# Esto es mejor hacerlo con un apply
for i in range(0,len(food_inspection_df)):
    food_inspection_df.loc[i, 'street_type'] = food_inspection_df.loc[i, 'address'].strip().split(' ')[-1]

In [26]:
food_inspection_df.state.unique()

array(['IL', nan, 'IN', 'NY', 'WI'], dtype=object)

In [25]:
columna = 'city'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

165

In [28]:
food_inspection_df.loc[food_inspection_df['city'] == 'chicago', 'state'] = 'il'

In [29]:
columna = 'state'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

50

# ZIP

In [44]:
columna = 'zip'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])
    
# filtering data  
# displayind data only with license = Not NaN  
print(len(food_inspection_df[bool_series]))
food_inspection_df[bool_series].head()

49


,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,longitude,location,violations,year,month,day,dayofweek,dayofyear,week,quarter
15,2492830,zea salon,zea salon,2775839,liquor,risk 3 (low),3707 n halsted st,NaN,il,NaN,...,-87.64940180063041,"{'latitude': '-87.64940180063041', 'longitude'...",NaN,2021,3,15,0,74,11,1
27,2492767,breakfast house restaurant,breakfast house restaurant,2776387,restaurant,risk 1 (high),3928-3932 n sheridan rd,NaN,il,NaN,...,-87.65466750153246,"{'latitude': '-87.65466750153246', 'longitude'...",NaN,2021,3,12,4,71,10,1
161,2492598,zea salon,zea salon,2775839,liquor,risk 3 (low),3707 n halsted st,NaN,il,NaN,...,-87.64940180063041,"{'latitude': '-87.64940180063041', 'longitude'...",5. procedures for responding to vomiting and d...,2021,3,9,1,68,10,1
936,2484790,lux galerie,lux galerie,2771243,restaurant,risk 1 (high),4012 s western ave,NaN,il,NaN,...,-87.68490231909283,"{'latitude': '-87.68490231909283', 'longitude'...","3. management, food employee and conditional e...",2021,2,11,3,42,6,1
1205,2484503,lux galerie,lux galerie,2771243,restaurant,risk 1 (high),4012 s western ave,NaN,il,NaN,...,-87.68490231909283,"{'latitude': '-87.68490231909283', 'longitude'...","3. management, food employee and conditional e...",2021,2,4,3,35,5,1


In [16]:
columna = 'zip'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  & pd.isnull(food_inspection_df['location'])
    
# filtering data  
# displayind data only with license = Not NaN  
print(len(food_inspection_df[bool_series]))
food_inspection_df[bool_series]

3


,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
92117,1763245,EAT N RUN CHICKEN AND FISH,EAT N RUN CHICKEN AND FISH,2442868,Restaurant,Risk 1 (High),,NaN,IL,NaN,2016-04-19T00:00:00.000,License,Not Ready,NaN,NaN,NaN,NaN
146362,1106878,FILIPINO AMERICAN COUNCIL,RIZAL CENTER,2225427,Golden Diner,Risk 1 (High),1332 W DRIVING PARK ROAD BSMT,NaN,IL,NaN,2013-09-26T00:00:00.000,Canvass,Out of Business,NaN,NaN,NaN,NaN
213091,60405,CASA CENTRAL LA POSADA,CASA CENTRAL LA POSADA,0,Restaurant,Risk 1 (High),,NaN,IL,NaN,2010-04-05T00:00:00.000,Canvass,Pass,NaN,NaN,NaN,37. TOILET ROOM DOORS SELF CLOSING: DRESSING R...


In [17]:
columna = 'zip'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  & pd.notnull(food_inspection_df['location'])
    
# filtering data  
# displayind data only with license = Not NaN  
print(len(food_inspection_df[bool_series]))
zip_notnull_df = food_inspection_df[bool_series]

46


In [18]:
def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']['postcode']


geolocator = Nominatim(user_agent='http' )

In [19]:
zipcodes = zip_notnull_df.apply(get_zipcode, axis=1, 
                                    geolocator=geolocator, 
                                    lat_field='latitude', lon_field='longitude')

In [20]:
food_inspection_df.loc[bool_series,'zip'] = zipcodes

# Inspection type

In [21]:
columna = 'inspection_type'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

1

In [22]:
columna = 'results'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

0

In [23]:
columna = 'violations'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

58087

# Location and latitude

In [157]:
columna = 'latitude'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

716

In [158]:
columna = 'longitude'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

716

In [165]:
columna = 'location'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna]) & pd.isnull(food_inspection_df['latitude']) & pd.isnull(food_inspection_df['longitude'] )
    
# filtering data  
# displayind data only with license = Not NaN  
print(len(food_inspection_df[bool_series]))
food_inspection_df[bool_series]

716


,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,latitude,longitude,location,year,month,day,dayofweek,dayofyear,week,quarter
852,2473009,jeni's splendid ice creams,jeni's splendid ice creams,2767260,restaurant,risk 2 (medium),2512 n milkwaukee ave,chicago,il,60647,...,NaN,NaN,NaN,2021,1,22,4,22,3,1
869,2472956,2096 n milwaukee llc,2096 n milwaukee llc,2767033,restaurant,risk 1 (high),2096 n milkwaukee ave,chicago,il,60647,...,NaN,NaN,NaN,2021,1,21,3,21,3,1
876,2472982,enzo & emilia,enzo & emilia,2762590,restaurant,risk 1 (high),2840-2542 n milkwaukee ave,chicago,il,60647,...,NaN,NaN,NaN,2021,1,21,3,21,3,1
882,2472978,enzo & emilia,enzo & emilia,2762588,restaurant,risk 1 (high),2840-2542 n milkwaukee ave,chicago,il,60647,...,NaN,NaN,NaN,2021,1,21,3,21,3,1
897,2472938,2096 n milwaukee llc,2096 n milwaukee llc,2767032,restaurant,risk 1 (high),2096 n milkwaukee ave,chicago,il,60647,...,NaN,NaN,NaN,2021,1,21,3,21,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216106,67840,tonti elementary,tonti elementary,25631,school,risk 1 (high),5815 s homan (3400w) ave,chicago,il,60629,...,NaN,NaN,NaN,2010,1,12,1,12,2,1
216189,158253,argentina foods,argentina foods,2009367,grocery store,risk 2 (medium),4500 s woods st,chicago,il,60609,...,NaN,NaN,NaN,2010,1,11,0,11,2,1
216302,67788,carson (old),carson (old),22601,school,risk 1 (high),5516 s maplewood (2532w) ave,chicago,il,60629,...,NaN,NaN,NaN,2010,1,7,3,7,1,1
216331,154225,chartwells,chartwells,1166638,special event,risk 2 (medium),2324 n fremont st,chicago,il,60614,...,NaN,NaN,NaN,2010,1,7,3,7,1,1


# Violations

In [32]:
len(food_inspection_df.violations.unique())

158360

In [85]:
columna = 'violations'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
pd.DataFrame(food_inspection_df[bool_series])

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2492939,JACK AND GINGERS,JACK AND GINGERS,1767508,Restaurant,Risk 1 (High),2048 W ARMITAGE AVE,CHICAGO,IL,60647,2021-03-16T00:00:00.000,Non-Inspection,No Entry,41.917829917196265,-87.67953075090323,"{'latitude': '-87.67953075090323', 'longitude'...",NaN
2,2492907,WHIPPED & BEATEN CULINARY WORKS,WHIPPED & BEATEN CULINARY WORKS,2776158,COOKING SCHOOL,Risk 3 (Low),3611-3617 N BROADWAY,CHICAGO,IL,60613,2021-03-16T00:00:00.000,License,Pass,41.94773174730859,-87.64734303203468,"{'latitude': '-87.64734303203468', 'longitude'...",NaN
3,2492904,WINTHROP CHILDREN,WNTHROP CHILDREN,2215541,Daycare (2 - 6 Years),Risk 1 (High),4848 N WINTHROP AVE,CHICAGO,IL,60640,2021-03-16T00:00:00.000,Canvass,Out of Business,41.97048400251602,-87.65780618896973,"{'latitude': '-87.65780618896973', 'longitude'...",NaN
15,2492830,ZEA SALON,ZEA SALON,2775839,Liquor,Risk 3 (Low),3707 N HALSTED ST,NaN,IL,60613,2021-03-15T00:00:00.000,License Re-Inspection,Pass,41.949326151061065,-87.64940180063041,"{'latitude': '-87.64940180063041', 'longitude'...",NaN
21,2492874,McDonald's,McDonald's,2776296,Restaurant,Risk 2 (Medium),500 W MADISON ST,CHICAGO,IL,60661,2021-03-15T00:00:00.000,License,Not Ready,41.88199433820508,-87.6397586848809,"{'latitude': '-87.6397586848809', 'longitude':...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217403,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601,2010-01-04T00:00:00.000,Tag Removal,Pass,41.88458626715456,-87.63101044588599,"{'latitude': '-87.63101044588599', 'longitude'...",NaN
217404,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.961605669949854,-87.67596676683779,"{'latitude': '-87.67596676683779', 'longitude'...",NaN
217405,52234,Cafe 608,Cafe 608,2013328,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.938006880423615,-87.6447545707008,"{'latitude': '-87.6447545707008', 'longitude':...",NaN
217406,70269,mr.daniel's,mr.daniel's,1899292,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.93844282365204,-87.76831838068422,"{'latitude': '-87.76831838068422', 'longitude'...",NaN


In [86]:
food_inspection_df.results.unique()

array(['No Entry', 'Pass', 'Out of Business', 'Pass w/ Conditions',
       'Fail', 'Not Ready', 'Business Not Located'], dtype=object)

In [53]:
violations = pd.DataFrame(food_inspection_df[bool_series]['violations']).reset_index().drop('index',axis=1)

In [54]:
violations

,violations
0,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ..."
1,40. PERSONAL CLEANLINESS - Comments: 2-402.11 ...
2,51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICE...
3,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...
4,40. PERSONAL CLEANLINESS - Comments: OBSERVED ...
...,...
159317,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
159318,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...
159319,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...
159320,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...


In [71]:
food_inspection_df['num_violations'] = len(food_inspection_df.loc[:,'violations'].split('|'))

AttributeError: 'Series' object has no attribute 'split'

In [88]:
type(food_inspection_df['violations'])

pandas.core.series.Series

In [90]:
f = lambda x: len(x['violations'].split('|')) if type(x['violations']) == str else x['violations']
food_inspection_df["num_violations"] = food_inspection_df.apply(f, axis=1)

In [94]:
food_inspection_df["num_violations"].mean()

4.475069356397736

In [96]:
food_inspection_df.loc[(food_inspection_df['results']=='Pass') & (pd.isnull(food_inspection_df['num_violations'])),
                      'num_violations'] = 0

In [99]:
food_inspection_df.loc[(food_inspection_df['results']=='Pass w/ Conditions') & (pd.isnull(food_inspection_df['num_violations'])),
                      'num_violations'] = 0

In [106]:
food_inspection_df.loc[(food_inspection_df['results']=='Fail') & (pd.isnull(food_inspection_df['num_violations'])),
                      ]

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations,num_violations
817,2484916,THE BATON SHOW LOUNGE,THE BATON SHOW LOUNGE,2732891,Restaurant,Risk 1 (High),4703-4713 N BROADWAY,CHICAGO,IL,60640,2021-02-18T00:00:00.000,License,Fail,41.9673635873824,-87.65856734441053,"{'latitude': '-87.65856734441053', 'longitude'...",NaN,NaN
1421,2484273,GODDESS ROCKS,GODDESS ROCKS,2590051,Catering,Risk 1 (High),1036 N KILBOURN AVE,CHICAGO,IL,60651,2021-01-29T00:00:00.000,Canvass Re-Inspection,Fail,41.89992381117936,-87.73868335454733,"{'latitude': '-87.73868335454733', 'longitude'...",NaN,NaN
1599,2473151,LIGHTHOUSE ARTSPACE CHICAGO,LIGHTHOUSE ARTSPACE CHICAGO,2767128,Restaurant,Risk 3 (Low),100-108 W GERMANIA PL,NaN,IL,60610,2021-01-26T00:00:00.000,License,Fail,41.91048443655356,-87.63168311419044,"{'latitude': '-87.63168311419044', 'longitude'...",NaN,NaN
2400,2472331,GODDESS ROCKS,GODDESS ROCKS,2590051,Catering,Risk 1 (High),1036 N KILBOURN AVE,CHICAGO,IL,60651,2021-01-06T00:00:00.000,Canvass,Fail,41.89992381117936,-87.73868335454733,"{'latitude': '-87.73868335454733', 'longitude'...",NaN,NaN
2436,2472355,TEQUILA CJ RESTAURANT,TEQUILA CJ RESTAURANT,2712258,Restaurant,Risk 1 (High),5750 S ARCHER AVE,CHICAGO,IL,60638,2021-01-06T00:00:00.000,License Re-Inspection,Fail,41.79675218648316,-87.75639619526761,"{'latitude': '-87.75639619526761', 'longitude'...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217307,98317,BREW & GROW,BREW & GROW,2013682,Grocery Store,Risk 3 (Low),3625-3629 N KEDZIE AVE,CHICAGO,IL,60618,2010-01-06T00:00:00.000,Consultation,Fail,41.94740242077963,-87.70769513166378,"{'latitude': '-87.70769513166378', 'longitude'...",NaN,NaN
217348,67756,LAGNIAPPE-A CREOLE CAJUN JOYNT,LAGNIAPPE-A CREOLE CAJUN JOYNT,1986108,RIVERWALK CAFE,Risk 2 (Medium),55 W RIVERWALK SOUTH,CHICAGO,IL,60601,2010-01-05T00:00:00.000,License Re-Inspection,Fail,41.88695199839048,-87.62987459293426,"{'latitude': '-87.62987459293426', 'longitude'...",NaN,NaN
217354,158243,DUNKIN DONUTS,NaN,2013344,Restaurant,Risk 2 (Medium),970 W PERSHING RD,CHICAGO,IL,60609,2010-01-05T00:00:00.000,License,Fail,41.82350359266238,-87.65066637652201,"{'latitude': '-87.65066637652201', 'longitude'...",NaN,NaN
217362,67760,MC INTOSH'S,MC INTOSH'S,1648623,NaN,Risk 1 (High),748 W 61ST ST,CHICAGO,IL,60621,2010-01-05T00:00:00.000,Out of Business,Fail,41.783502980089736,-87.64438637632864,"{'latitude': '-87.64438637632864', 'longitude'...",NaN,NaN
